In [ ]:
import rclpy as rp
import jupyros.ros2 as jr2
import turtle_sim as turtle
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from time import time
import os
from std_msgs.msg import String
from geometry_msgs.msg import Twist
from sidecar import Sidecar
from time import time, sleep
import math



In [ ]:
# Initialize ROS communications for a given context
if(rp.ok() == False):
    rp.init()
else:
    print("rclpy already initiated")

In [ ]:
superturtle = rp.create_node("superturtle")
key = rp.create_node("keyInput")
com = rp.create_node("com")
simple = rp.create_node("simple")
translate = rp.create_node("translate")

In [ ]:
pub_node = com.create_publisher(Twist,'/cmd_vel',10)


In [ ]:
turtlesim = turtle.TurtleSim(background_color="#0000FF")
display(turtlesim.canvas)

In [ ]:
def cb(msg):
    
    key = msg.data
    
    ## General Keybindings 
    ## "key: (x, y, z, a (alpha), b (beta), g (gamma)"
    
    moveBindings = {
    'ArrowLeft': (0,0,0,1),
    'ArrowRight':(0,0,0,-1),
    'ArrowUp': (1,0,0,0),
    'ArrowDown': (-1,0,0,0)
    }

    """
    'i': (1, 0, 0, 0, 0, 0),
    'o': (1, 0, 0, -1, 0, 0)),
    'j': (0, 0, 0, 1, 0, 0)),
    'l': (0, 0, 0, -1, 0, 0)),
    'u': (1, 0, 0, 1, 0, 0)),
    ',': (-1, 0, 0, 0, 0, 0)),
    '.': (-1, 0, 0, 1, 0, 0)),
    'm': (-1, 0, 0, -1, 0, 0)),
    'O': (1, -1, 0, 0, 0, 0)),
    'I': (1, 0, 0, 0, 0, 0)),
    'J': (0, 1, 0, 0, 0, 0)),
    'L': (0, -1, 0, 0, 0, 0)),
    'U': (1, 1, 0, 0, 0, 0)),
    '<': (-1, 0, 0, 0, 0, 0)),
    '>': (-1, -1, 0, 0, 0, 0)),
    'M': (-1, 1, 0, 0), 0, 0),
    't': (0, 0, 1, 0, 0, 0)),
    'b': (0, 0, -1, 0, 0, 0)),
    """


    
    
    ## General Keybinding Decoder
    if key in moveBindings.keys():
        x = moveBindings[key][0]
        y = moveBindings[key][1]
        z = moveBindings[key][2]
        a = moveBindings[key][3]
        b = moveBindings[key][4]
        g = moveBindings[key][5]
        
    else:
        x = 0.0
        y = 0.0
        z = 0.0
        a = 0.0
        b = 0.0
        g = 0.0
   
    twist = Twist()
    twist.linear.x = x
    twist.angular.z = g 

    pub_node.publish(twist)


    return 0

**TIP:** When using JupyterLab, you can right-click on the canvas and select *Create New View from Output*

In [ ]:
# Canvas default size is 1600x1600 starting from top-left corner
turtlesim.spawn(name="simon", pose={"x": 600, "y": 350, "theta": 20})

turtlesim.spawn(pose={"x": 1050, "y": 1250, "theta": 90})

# Change the turtle path color
turtlesim.turtles["turtle1"].path_color = "#F2F2F2"

Open two terminals and run:

```sh
rosrun turtlesim turtlesim_node
rosrun turtlesim turtle_teleop_key
```

### Subscriber

In [ ]:
poses = {}

for name in turtlesim.turtles.keys():
    poses[name] = turtlesim.turtles[name].pose
    
print(poses["turtle1"])

In [ ]:
topic_name = '/turtle1/pose'

# Based on the dimensions of window controlled by teleop keyboard
def convert_xy_units(x, y):
    convert_factor = 11.08
    x_convert = x / convert_factor * turtlesim.canvas.width
    y_convert = (convert_factor - y) / convert_factor * turtlesim.canvas.height
    
    return x_convert, y_convert



def callback_move_turtles(msg):
    name = "turtle1"
    x, y = convert_xy_units(msg.x, msg.y)
    poses[name] = {"x": x,
                   "y": y,
                   "theta": msg.theta}
    
    turtlesim.move_turtles(new_poses=poses)

jr2.subscribe(topic_name, Pose, lambda msg: callback_move_turtles(msg))

### Spiral Publisher

In [ ]:
run = True

In [ ]:
%%thread_cell


pub = rospy.Publisher(topic_name, Pose, queue_size=10, latch=True)
i = 0

while run:
    msg = Pose()
    msg.x = (i/1800*5)*math.sin(i / 180 * math.pi) + 11.08 / 2
    msg.y = (i/1800*5)*math.cos(i / 180 * math.pi) + 11.08 / 2
    msg.theta = - i / 180 * math.pi
    pub.publish(msg)
    rate.sleep()
    i += 1
print("Done")

In [ ]:
run = False

## Spawn Service and Client

In [ ]:
def spawn_turtle(msg):
    print(f"Spawning turtle at x={msg.x} and y={msg.y}")
    turtlesim.spawn(name=msg.name, 
                    pose={"x": msg.x, 
                          "y": msg.y, 
                          "theta": msg.theta})
    return "Spawned"

In [ ]:
%%thread_cell

srv = rospy.Service('spawn_srv', Spawn, spawn_turtle)

In [ ]:
jupyros.client('spawn_srv', Spawn)